In [1]:
import numpy as np
import cv2
import pickle
    
face_cascade=cv2.CascadeClassifier('Cascades\haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0) #Capture image from default camera
cap.set(3, 640) # set video widht
cap.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cap.get(3)
minH = 0.1*cap.get(4)

recognizer=cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

labels={}
with open("labels.pickle",'rb') as f:
    labels=pickle.load(f)
    labels={v:k for k,v in labels.items()}

#While loop for streaming the data
while (cap.isOpened()):
    ret, frame = cap.read() 
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #converting to grayscale
    faces=face_cascade.detectMultiScale(gray,scaleFactor=1.5,minNeighbors=5,minSize = (int(minW), int(minH)),)
    
    for (x,y,w,h) in faces:
        #print(x,y,w,h)
        roi_gray=gray[y:y+h, x:x+w]
        roi_color=frame[y:y+h, x:x+w]
        
        id_, confidence = recognizer.predict(roi_gray)
        # Check if confidence is less than 100 ==> "0" is perfect match 
        if (confidence < 100):
            name = labels[id_]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            name = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
            
        font=cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, name, (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(frame, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (128,0,128), 2)
        
    cv2.imshow('frame',frame)
        
    if cv2.waitKey(20) & 0xFF == 27: #Press ESC to quit
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#References
#[1] CodingEntrepreneurs, YouTube, 11-Apr-2018. [Online]. Available: https://www.youtube.com/watch?v=PmZ29Vta7Vc&list=WL&index=35&t=0s.
#[2] https://www.hackster.io/mjrobot/real-time-face-recognition-an-end-to-end-project-a10826